## Various Number Theory Experiments

_burt rosenberg
<br>
4 november 2019_

Topics:

- [Congruence classes](#Congruence-classes)
- [The divisor principle ideal](#ideal)
- [Extended gcd algorithm](#Extended-gcd-algorithm)
- [The multiplication table](#multiplication)
- [The self-map](#homothety)
- [Euler Phi Function and Little Fermat](#euler-phi)
- [Wilson's theorem](#wilsons)
- [Primality Testing](#Primality-Testing)


### Congruence classes

In [79]:
import random

#
# begin with a equivalence on Z defined by an n, and a=b when
# n|a-b. this creates n equivalence classes, 
#    { i+j*n | j is any integer } i = 0,1,...,n-1
# the equivalence class can be thought of in 4 ways, 
#    1) by the est (above)
#    2) by any integer in the set
#    3) by the least non-negative integer in the set (the residue)
#    4) in a signed fashion with the n integers closest to 0,
#       both positive and negative
# in (4) there is no ambiguitiy for when n is even, for then
# n/2 is both positive and negative.
# 
# to define an arithmetic operation, it is sufficient to show 
# this in the interpretation (2), using sample integers from the
# equivalence classes (henceforth also - congruence classes).
# that is, A+B=C is defined as taking any a from A, b from B, 
# and finding the unique C with member a+b.
#
# it is an easy enough proof. however the following is an experiment
# to help conceptualize the proof.
#


def congruence_add_p(A,B,C):
    """
    does A + B = C, where A, B and C are congruence classes
    """
    s = random.choice(A[:5])+random.choice(B[:5])
    return s in C

def build_congruence(n):
    c_g = [i for i in range(n)]
    for i in range(n):
        c_g[i] = [i+j for j in range(0,15*n,n)]
    return c_g

def test_congruence_add_p(n):
    c_g = build_congruence(n)
    for i in range(n):
        for j in range(n):
            for k in range(3):
                if not congruence_add_p(c_g[i],c_g[j],c_g[(i+j)%n]):
                    print("***fail***")
        for i in range(n):
            for j in range(n):
                for k in range(3):
                    if congruence_add_p(c_g[i],c_g[j],c_g[(i+j+k+1)%n]):
                        print("***fail***")

    print("***pass***")
    
test_congruence_add_p(5)

***pass***


<a id="ideal"></a>
### The divisor principle ideal

In [80]:
#
# the set { i*a + j*b | for i and j all integers} is the
# same as the set { gcd(a,b)*i | for i all integers }
# 
# this is useful in finding a gcd, as once one can start at the
# two elements of this set, a and b, and "climb down" the ladder
# of integers in the set until one arrives at the rung just above
# zero. that is the gcd.
#

def integer_span(a,b,n,width=10):
    C = []
    for i in range(-n,n):
        for j in range(-n,n):
            d = i*a+j*b
            if d not in C:
                C.append(d)
    C = sorted(C)
    return [i for i in filter(lambda x: x<width and x>-width, C)]


print(integer_span(7,9,5))
    
    

[-9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


### Extended gcd algorithm

In [81]:
#
# using the bezouts set, not only can we find the gcd
# but we can find the promised s and t where,
#   gcd(a,b) = s*a + b*t
# as we work down the bezouts set towards zero, but jumping
# from remainder to remainder, we keep track of the quotients
# involved in the remainder calculation.
#


def extended_gcd(a,b):
    """
    extended GCD algorithm. recursive.
    returns (d,s,t) where d = s*a+t*b 
    and d = gcd(a,b)
    """
    assert(
        a>=0 and b>=0 )
    if b==0:
        return (a,1,0)
    (q,r) = divmod(a,b)
    (d,s,t) = extended_gcd(b,r)
    # gcd(a, b) == gcd(b, r) == s*b + t*r == s*b + t*(a - q*b)
    return (d,t,s-q*t)


def test_e_gcd(n):
    for i in range(n):
        (d,s,t) = extended_gcd(i,n)
        if d==1:
            # check the inverse property
            if (i*s%n)!=1:
                print("***failed***")
                return
        else:
            # check the divisibility property
            if i%d!=0 or n%d!=0:
                print("***failed***")
                return
    print("***passed***")

    
def invertibles(n):
    xr = [i for i in 
          filter(lambda x: (extended_gcd(x,n)[0]==1),
                 range(1,n))]
    xnr = [i for i in filter(lambda x: (x not in xr),
                 range(n))]
    return xr, xnr

            
test_e_gcd(100000)


***passed***


<a id="multiplication"></a>
### The multiplication table

In [82]:
#
# restricted to X^*_n subset of X_n, there is a multiplication
# table, with unique inverses. in fact, each row and col of the 
# multiplication table is a permutation of the residue classes
#

def mult_table(n):
    return [[(i*j%n) for j in range(n)] for i in range(n)]

def print_mult_table(n):
    t = mult_table(n)
    print("mult",end="")
    for col in invertibles(n)[0]:
        print("\t",col,end="")
    print("")
    for col in invertibles(n)[0]:
        print("\t----",end="")
    for row in invertibles(n)[0]:
        print("")
        print(row,end="")
        for col in invertibles(n)[0]:
            print("\t",t[row][col],end="")
    print("")
    return
    
    
print_mult_table(10)

mult	 1	 3	 7	 9
	----	----	----	----
1	 1	 3	 7	 9
3	 3	 9	 1	 7
7	 7	 1	 9	 3
9	 9	 7	 3	 1


<a id="homothety"></a>
### The self-map

In [109]:
#
# the approach here is to understand the map of x goest to
# g x, where g is an invertible element in Zn. 
#
# this is a permutation, even on the non-invertible elements,
# and when expressed in cycle notation, the kernel of the map
# is the orbit of g in mod n, <g>.
#
# the other cycles trace out the co-sets of the kernel, and
# we note Legrange property a subgroup, divides the size of 
# the containing group.
#
# when the self map is restricted to the invertibles, it 
# forms the basis for the proof of Euler's/Little Fermat.
# it also introduces the euler totient phi function
#

def generalized_orbit(g,n):
    """
    The generalized orbit of g mod n is the permutation on Zn 
    multiplication by g, x goes to x*g%n.
    """
    o = [1]
    if extended_gcd(g,n)[0]!=1:
        return o
    if g!=1:
        o += [g]
        while (o[-1]*g)%n!=1:
            o += [(o[-1]*g)%n]
    O = [o[:]]
    
    def flatten(O):
        l = []
        for o in O:
            l += o
        return l

    xr, xnr = invertibles(n)
    for l in range(0,len(xr)//len(o)):
        for x in xr:
            if x not in flatten(O):
                O += [[j*x%n for j in o]]
    return O

def visualize_orbit(n,kernel_size_order=False):
    xr, xrn = invertibles(n)
    #print("inv:",xr)
    
    print("\n\x1b[32m[ 1\x1b[31m gen\x1b[32m kern ]\x1b[0m\t[ coset ] [ coset ] ...")

    g_o = []    
    for g in sorted(xr):
        g_o += [generalized_orbit(g,n)]
    if kernel_size_order:
        g_o = [y for (x,y) in sorted([(len(x),x) for x in g_o],reverse=False)]

    for g in g_o:
        if len(g[0])==1:
            continue
        else: 
            print("\x1b[32m[ ",end="")
            i = 0
            for ge in g[0]:
                if i==1:
                    print("\x1b[31m",ge,"\x1b[32m",end="")
                else:
                    print(ge,"",end="")
                i += 1
            print("]\x1b[0m  \t",end="")
        for ge in g[1:]:
            print(ge," ",end="")

        print()
    return 

def noninvt(n):
    xn, xnr = invertibles(n)
    print("\ninvertible times an non-invertible")
    for x in xn:
        print(x,[i*x%n for i in xnr])
    print("\nproduct of non-invertibles")
    for x in xnr:
        if x==0:
            continue
        print(x,[i*x%n for i in xnr])
        
visualize_orbit(15,kernel_size_order=False)
visualize_orbit(25,kernel_size_order=False)
visualize_orbit(11,kernel_size_order=True)
visualize_orbit(23,kernel_size_order=True)


[ 1 gen kern ]	[ coset ] [ coset ] ...
[ 1  2 4 8 ]  	[7, 14, 13, 11]  
[ 1  4 ]  	[2, 8]  [7, 13]  [11, 14]  
[ 1  7 4 13 ]  	[2, 14, 8, 11]  
[ 1  8 4 2 ]  	[7, 11, 13, 14]  
[ 1  11 ]  	[2, 7]  [4, 14]  [8, 13]  
[ 1  13 4 7 ]  	[2, 11, 8, 14]  
[ 1  14 ]  	[2, 13]  [4, 11]  [7, 8]  

[ 1 gen kern ]	[ coset ] [ coset ] ...
[ 1  2 4 8 16 7 14 3 6 12 24 23 21 17 9 18 11 22 19 13 ]  	
[ 1  3 9 2 6 18 4 12 11 8 24 22 16 23 19 7 21 13 14 17 ]  	
[ 1  4 16 14 6 24 21 9 11 19 ]  	[2, 8, 7, 3, 12, 23, 17, 18, 22, 13]  
[ 1  6 11 16 21 ]  	[2, 12, 22, 7, 17]  [3, 18, 8, 23, 13]  [4, 24, 19, 14, 9]  
[ 1  7 24 18 ]  	[2, 14, 23, 11]  [3, 21, 22, 4]  [6, 17, 19, 8]  [9, 13, 16, 12]  
[ 1  8 14 12 21 18 19 2 16 3 24 17 11 13 4 7 6 23 9 22 ]  	
[ 1  9 6 4 11 24 16 19 21 14 ]  	[2, 18, 12, 8, 22, 23, 7, 13, 17, 3]  
[ 1  11 21 6 16 ]  	[2, 22, 17, 12, 7]  [3, 8, 13, 18, 23]  [4, 19, 9, 24, 14]  
[ 1  12 19 3 11 7 9 8 21 2 24 13 6 22 14 18 16 17 4 23 ]  	
[ 1  13 19 22 11 18 9 17 21 23 24 12 6 3 

<a id="euler-phi"></a>
### Euler Phi Function and Little Fermat

In [106]:
#
# the euler phi function of n counts the number of elements
# relativey prime to n in the range 1 to n-1. there is a formula
# to efficiently calculate phi, but here we content ourselves 
# to make a list of all invertibles in Z_n and phi is the number
# of elements on the list.
#
# because we know Z_n goes to Z_n as a permutation by the action
# multiplication by an invertible, we get Euler's Theorem, which
# is a generalization of Little Fermat to composite moduli.
#
# also noted is wilson's theorem, also a consequence of this
# self-action, which concerns the factorial (n-1)! in Z_n.
#

def euler_phi_function(n):
    """
    phi(n) = n Prod (1-1/p), all primes p|n.
    """
    return len(invertibles(n)[0])

def proof_of_eulers_theorem(n):
    """
    Euler's is a generalization of little fermat for 
    any n. its proof can be that the map Zn->Zn multiplication
    by a where a is rel prime to n, is a permutation.
    
    Little fermat is the case n is a prime.
    """
    xn, xnr = invertibles(n)
    # phi = len(xn)
    phi = euler_phi_function(n)
    for x in xn:
        p = [x*i%n for i in xn]
        if sorted(p)!=sorted(xn):
            print("***fail***")
        if pow(x,phi,n)!=1:
            print("***fail***")
    print("***passed fermat test***")

def generalized_orbit_structure(n):
    n = euler_phi_function(n)
    print("group size:",n)
    for i in range(2,n+1):
        if n%i==0:
            print(euler_phi_function(i),"kernels of size",i)

proof_of_eulers_theorem(113)
generalized_orbit_structure(7)
generalized_orbit_structure(11)
generalized_orbit_structure(23)
generalized_orbit_structure(15)

***passed fermat test***
group size: 6
1 kernels of size 2
2 kernels of size 3
2 kernels of size 6
group size: 10
1 kernels of size 2
4 kernels of size 5
4 kernels of size 10
group size: 22
1 kernels of size 2
10 kernels of size 11
10 kernels of size 22
group size: 8
1 kernels of size 2
2 kernels of size 4
4 kernels of size 8


<a id="wilsons"></a>
### Wilsons theorem

In [85]:
def wilsons_theorem(n):
    """
    Gauss proved the generalization that 
    the product of all numbers relatively
    prime to n between 1 and n-1, 
    is -1 in the cases 
    - the power of an odd prime
    - twice such a number
    - or 4
    and 1 in all other cases.
    """
    xn, xnr = invertibles(n)
    p = 1
    for x in xn:
        p = (p*x)%n
    if p==(n-1):
        p = -1
    if p==1:
        print("***not a prime***")
    elif p==-1:
        print("***a prime power, twice a prime power, or 4***")
    else:
        print("***fail***")          

wilsons_theorem(113)
        

***a prime power, twice a prime power, or 4***


### Primality Testing

In [111]:
#
# in cases we need a prime number, we must have an efficient
# way of determining if a number is prime. trial divisor or
# the Sieve of Eratosthenes is not efficient.
#
# using the converse of Little Fermat is an idea, but fails because
# of Carmichael numbrers. Miller-Rabin is similar, but does not fail.
#
# these are PPT algorithms, that express their randomness in the
# possibility of a one-sided error. The algorithms randomly draw
# from Z_n to see if it is a witness to compositeness. There are
# no witnesses to primality, except that repeated trials has
# failed to find a witness to compositeness.
#
# the key to miller rabin is a conductor to 1, by doing the
# exponentiation of little fermat several steps. it will lead
# necessarily to one in a certain way for a prime, but not
# necessarilty in that one for a composite
#
def miller_rabin_conductor(x,n,more_squarings=0):
    if (n%2==0):
        return [0]
    assert(x>0)
    if x==1:
        return [1]
    s = n-1
    t = 0
    while s%2==0:
        s = s//2
        t += 1
    assert(s%2!=0)
    assert( (n-1)==pow(2,t)*s)
    l = [x, pow(x,s,n)]
    for i in range(t+more_squarings):
        if (l[-1]==1):
            return l
        l.append(pow(l[-1],2,n))
    return l

def is_miller_rabin_witness(x,n,verbose=False):
    """
    test is l[1] is either 1 or l[i]==-1 for some i>0  
    """
    mc = miller_rabin_conductor(x,n)
    if verbose: print(mc)
    if mc[1]==1 or any(filter(lambda z: z==n-1, mc[1:])):
        return False
    return True

def miller_rabin(n,trials=1000,verbose=False):
    if n==2:
        return True
    if n%2==0:
        return False
    # if n is a pure power return false
    for i in range(trials):
        w = random.randint(2,n-2)
        if is_miller_rabin_witness(w,n,verbose):
            return False
    return True


print("for a prime:")
print("trying to find a witness to deny primality...")
print(miller_rabin(37,trials=15,verbose=True))

print("\nfor a composite:")
print("finding a witness that denies primality...")
print(miller_rabin(35,trials=15,verbose=True))

for a prime:
trying to find a witness to deny primality...
True

for a composite:
finding a witness that denies primality...
False
